In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import networkx as nx

In [2]:
data = pd.read_csv("rabobank_data.csv", header = None, delimiter = ';')
data = data.rename(columns=data.iloc[0])
data = data.drop(0)
data['count'] = data['count'].astype(int)
data['total'] = data['total'].astype(int)

C:\Users\Dev\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (1,2,3,4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
def map_account_to_numbers():
    lst1 = np.array(data['start_id'])
    lst2 = np.array(data['end_id'])
    lst = np.concatenate((lst1,lst2))
    lst = np.unique(lst)

    dict = {}

    for i in range(len(lst)):
        dict[lst[i]] = i + 1
    return dict

mapping = map_account_to_numbers()

In [4]:
def helper(node, step):
    if(step == 1):
        return np.array([node])
    
    lst = [node]
    lst = np.array(lst)
    
    conn = np.array(data[data['start_id'] == node]['end_id'])
    lst  = np.concatenate((lst,conn))
    
    for n in conn:
        ret = helper(n,step-1)
        lst = np.concatenate((lst,ret))
    
    return lst

def snowboard_sampling(step):
    node = np.random.choice(data['start_id'], 1)[0]
    lst = helper(node,step)
    return np.unique(lst)

node_in_use = snowboard_sampling(2)

In [6]:
data_temp = data[(data['start_id'].isin(node_in_use))]
temp = []

for i in data_temp.index:
    start_id = data_temp['start_id'][i]
    end_id = data_temp['end_id'][i]
    wgt = data_temp['count'][i]
    src = mapping[start_id]
    dst = mapping[end_id]
    temp.append([src,dst,wgt])

In [7]:
edge = pd.DataFrame(data=temp, columns=['Source','Target','Weight'])
edge.to_csv('edge.csv', sep = ',', index = False)